<a href="https://colab.research.google.com/github/harshal-ugale/S3_Bucket/blob/master/day3_gen_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Running or Importing .py Files with Google Colab

Run these codes first in order to install the necessary libraries and perform authorization.

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Now you can reach you Google Drive with:

In [2]:
!ls '/content/drive'

'My Drive'


copying the utilities and audio_path files to the 'jw' folder

In [0]:
!cp -r /content/drive/'My Drive'/utilities /content/jw/

In [4]:
!cp -r /content/drive/'My Drive'/validated_audio_data /content/jw/

^C


In [5]:
!cp -r /content/drive/'My Drive'/validated_audio_data/sales_validated /content/jw/validated_audio_data

^C


installing virtual enviorment

In [6]:
!pip3 install virtualenv
!virtualenv theanoEnv

     |████████████████████████████████| 3.4MB 6.3MB/s 
Using base prefix '/usr'
New python executable in /content/theanoEnv/bin/python3
Also creating executable in /content/theanoEnv/bin/python
Installing setuptools, pip, wheel...
done.


creating virtual enviorment and activating the enviorment

In [7]:
!virtualenv /content/Ds   #here deepspeech is edited to Ds.
!source /content/Ds/bin/activate

Using base prefix '/usr'
New python executable in /content/Ds/bin/python3
Also creating executable in /content/Ds/bin/python
Installing setuptools, pip, wheel...
done.


installing deepspeech and upgrading it, if not upgraded.

In [8]:
!pip3 install deepspeech-gpu
!pip3 install --upgrade deepspeech

     |████████████████████████████████| 18.7MB 424kB/s 
     |████████████████████████████████| 9.6MB 5.9MB/s 


clone the deepspeech repository

In [9]:
!git clone https://github.com/mozilla/DeepSpeech

Cloning into 'DeepSpeech'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 14805 (delta 29), reused 31 (delta 13), pack-reused 14740
Receiving objects: 100% (14805/14805), 46.39 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (9801/9801), done.


installing requirement dependencies using pip3

In [1]:
%cd DeepSpeech
!pip3 install -r requirements.txt

/content/DeepSpeech


install ds_ctcdecoder python package. ds_ctcdecoder is used to decode deepspeech model output(audio) to text.

In [2]:
!pip3 install $(python3 util/taskcluster.py --decoder)

     |████████████████████████████████| 5.6MB 377kB/s 


train data

In [8]:
! python /content/jw/utilities/support_scripts/createCSV.py --audio_path "/content/jw/validated_audio_data/non_sales_validated" --csv_output_path '/content/jw/data1/train/'  --csv_file_name 'train.csv'

totalFiles=226
Successfully created CSV file train.csv at /content/jw/data1/train/


test data

In [9]:
! python /content/jw/utilities/support_scripts/createCSV.py --audio_path "/content/jw/validated_audio_data/sales_validated/test_data/" --csv_output_path '/content/jw/data1/test/'  --csv_file_name 'test.csv'

totalFiles=32
Successfully created CSV file test.csv at /content/jw/data1/test/


dev data

In [10]:
! python /content/jw/utilities/support_scripts/createCSV.py --audio_path "/content/jw/validated_audio_data/sales_validated/dev_data/" --csv_output_path '/content/jw/data1/dev/'  --csv_file_name 'dev.csv'

totalFiles=72
Successfully created CSV file dev.csv at /content/jw/data1/dev/


generateLMtext.py file 

In [11]:
!python /content/jw/utilities/support_scripts/generateLMText.py --audio_path "/content/jw/validated_audio_data/" --lm_output_path '/content/jw/lm_word/' --lm_file_name 'word.txt'

totalFiles=330
Successfully created lm file word.txt at /content/jw/lm_word/


#kenLM

In [12]:
%cd native_client
%rm -rf kenlm \
    && git clone --depth 1 https://github.com/kpu/kenlm && cd kenlm \
    && mkdir -p build \
    && cd build \
    && cmake .. \
    && make -j 4


/content/DeepSpeech/native_client
Cloning into 'kenlm'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (300/300), done.
remote: Total 312 (delta 9), reused 73 (delta 4), pack-reused 0
Receiving objects: 100% (312/312), 483.90 KiB | 1.26 MiB/s, done.
Resolving deltas: 100% (9/9), done.
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pth

#Generate words.arpa

In [13]:
!/content/DeepSpeech/native_client/kenlm/build/bin/lmplz --text /content/jw/utilities/vocab.txt  --arpa /content/jw/arpa/words.arpa --o 5 --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /content/jw/utilities/vocab.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1923358720 bytes == 0x55b7fe6e0000 @  0x7f6146c3a1e7 0x55b7fd518932 0x55b7fd4ad188 0x55b7fd48c016 0x55b7fd47817e 0x7f6144dd3b97 0x55b7fd479bca
tcmalloc: large alloc 8975663104 bytes == 0x55b871122000 @  0x7f6146c3a1e7 0x55b7fd518932 0x55b7fd50358a 0x55b7fd503fa8 0x55b7fd48c033 0x55b7fd47817e 0x7f6144dd3b97 0x55b7fd479bca
****************************************************************************************************
Unigram tokens 20364 types 1693
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:20316 2:1065772288 3:1998323072 4:3197316864 5:4662753792
tcmalloc: large alloc 4662755328 bytes == 0x55b7fe6e0000 @  0x7f6146c3a1e7 0x55b7fd518932 0x55b7fd50358a 0x55b7fd503fa8 0x55b7fd48c5dd 0x55b7fd47817e 0x7f6144dd3b97 0x55b7fd479bca
tcmalloc: large alloc 1998323712 by

#Genrate lm.binary from words.arpa

In [14]:
!/content/DeepSpeech/native_client/kenlm/build/bin/build_binary -T -s /content/jw/arpa/words.arpa /content/jw/lmbinary/lm.binary

Reading /content/jw/arpa/words.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


#Generating generate_trie floder using taskcluster.py

In [15]:
!python3 /content/DeepSpeech/util/taskcluster.py --target /content/DeepSpeech/native_client/

Downloading: 100%

generate_trie
libdeepspeech.so
LICENSE
deepspeech
deepspeech.h
README.mozilla


#Generating trie file using alphabet.txt and lm.binary

In [16]:
!/content/DeepSpeech/native_client/generate_trie /content/jw/utilities/lm_text/alphabet.txt /content/jw/lmbinary/lm.binary /content/jw/trie/trie

ERROR: AlignOutput: Can't determine stream position
ERROR: Could not align file during write after header


Training the model

In [18]:
%cd ..

/content/DeepSpeech


In [19]:
!./DeepSpeech.py --train_files /content/jw/data1/train/train.csv --dev_files /content/jw/data1/dev/dev.csv --test_files /content/jw/data1/test/test.csv --epochs 3 --dropout_rate 0.2

/bin/sh: 1: sox: not found
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
W1204 10:37:52.097349 139959306016640 deprecation.py:323] From /usr/local/lib/python3.6/dist-

In [20]:
!pip3 list

Package                  Version     
------------------------ ------------
absl-py                  0.8.1       
alabaster                0.7.12      
albumentations           0.1.12      
altair                   3.2.0       
astor                    0.8.0       
astropy                  3.0.5       
atari-py                 0.2.6       
atomicwrites             1.3.0       
attrdict                 2.0.1       
attrs                    19.3.0      
audioread                2.1.8       
autograd                 1.3         
Babel                    2.7.0       
backcall                 0.1.0       
backports.tempfile       1.0         
backports.weakref        1.0.post1   
bcrypt                   3.1.7       
beautifulsoup4           4.6.3       
bleach                   3.1.0       
blis                     0.2.4       
bokeh                    1.0.4       
boto                     2.49.0      
boto3                    1.10.18     
botocore                 1.13.18     
Bottleneck  